In [299]:
import numpy as np
import cantera as ct
import matplotlib.pyplot as plt

In [300]:
# import the model for gas-Fe interface and adjacent gas phase
cat = ct.Interface('FeSiO2_NOMC.yaml','Fe_surf')
gas = cat.adjacent['gas']

# Modify gas phase reaction rates
for i in range(gas.n_reactions):
    gas.set_multiplier(0.001, i)

# Modify gas phase reaction rates
for i in range(cat.n_reactions):
    cat.set_multiplier(1, i)

In [301]:
# Set the initial conditions
T0 = 1373.15 # K
p0 = ct.one_atm # atm
gas.TPX = T0, p0, "CH4: 0.9, N2: 0.1"
cat.TP = T0, p0
D = 0.004 # diameter of the tube [m]
Ac = np.pi * D**2 / 4 # corss section of the tube [m2]
w_cat = 0.82 # weight of catalyst [gcat]
GHSV = 0.002016 # m3/gcat/hr
u0 = GHSV * w_cat /3600 / Ac # [m/s]

In [302]:
# Set the Reactor conditions
reactor = ct.FlowReactor(gas)
reactor.area = Ac
reactor.mass_flow_rate = gas.density * u0 * Ac
reactor.energy_enabled = False

rsurf = ct.ReactorSurface(cat, reactor)
net = ct.ReactorNet([reactor])
soln = ct.SolutionArray(gas, extra=['x', 'surf_coverages'])


In [303]:
# Integrate the reactor network
while net.distance < 0.06:
    net.step()
    wdot = rsurf.kinetics.net_production_rates
    soln.append(TDY=reactor.thermo.TDY,
                x=net.distance,
                surf_coverages=rsurf.coverages
                )

In [304]:
print(soln('CH4').Y[-1][0])

0.8348085008297097
